In [1]:
import re
from itertools import combinations
from copy import deepcopy

In [2]:
data = open('input/12.txt').read().splitlines()

steps = 1000

## Extract moons

In [3]:
moons = []
pattern = r'<x=(.*), y=(.*), z=(.*)>'
for idx, m in enumerate(data):
    match = re.match(pattern, m)
    x, y, z = list(map(int, match.groups()))
    moons.append([[x, y, z], [0, 0, 0]])
original_moons = deepcopy(moons)

## Perform 1 iteration

In [4]:
def run_iteration():
    for idx1, idx2 in combinations(range(len(moons)), 2):
        # Calculate velocity
        pos1, vel1 = moons[idx1]
        pos2, vel2 = moons[idx2]
        for a in range(3):
            if pos1[a] > pos2[a]:
                vel1[a] -= 1
                vel2[a] += 1
            elif pos1[a] < pos2[a]:
                vel1[a] += 1
                vel2[a] -= 1
        moons[idx1][1] = vel1
        moons[idx2][1] = vel2
    
    # Update moons
    for idx, moon in enumerate(moons):
        pos = [moon[0][0] + moon[1][0], moon[0][1] + moon[1][1], moon[0][2] + moon[1][2]]
        vel = moon[1]
        moons[idx] = [pos, vel]

# Part 1

In [5]:
for i in range(steps):
    run_iteration()

In [6]:
def calculate_energy(moon):
    pos, vel = moon
    potential = sum(list(map(abs, pos)))
    kinetic = sum(list(map(abs, vel)))
    return potential * kinetic
    
energies = sum(list(map(calculate_energy, moons)))
print(f'Answer #1: {energies}')

Answer #1: 7138


# Part 2
Each axle moves independently of the others.  
So for each axle check when they repeat again.
When that is done, calculate lowest common multiplier, when they will repeat again together.

In [7]:
def compare_state_to_initial():
    x, y, z = [], [], []
    for moon_idx, (pos1, vel1) in enumerate(moons):
        pos2, vel2 = original_moons[moon_idx]
        
        x.append(pos1[0] == pos2[0] and vel1[0] == vel2[0])
        y.append(pos1[1] == pos2[1] and vel1[1] == vel2[1])
        z.append(pos1[2] == pos2[2] and vel1[2] == vel2[2])
        
    return sum(x) == len(x), sum(y) == len(y), sum(z) == len(z)        

## Run iterations until repeats

In [8]:
moons = deepcopy(original_moons)

result = [False, False, False]
cntr = 0
while result.count(False) > 0:
    cntr += 1
    run_iteration()
    x, y, z = compare_state_to_initial()
    if x and not result[0]:
        result[0] = cntr
    if y and not result[1]:
        result[1] = cntr
    if z and not result[2]:
        result[2] = cntr

## Calculate lowest common multiple of the axles

In [9]:
# From https://gist.github.com/endolith/114336/eff2dc13535f139d0d6a2db68597fad2826b53c3
def gcd(a,b):
    """Compute the greatest common divisor of a and b"""
    while b > 0:
        a, b = b, a % b
    return a
    
def lcm(a, b):
    """Compute the lowest common multiple of a and b"""
    return int(a * b / gcd(a, b))

In [10]:
x, y, z = result
answer_2 = lcm(lcm(x, y), z)
print(f'Answer #2: {answer_2}')

Answer #2: 572087463375796
